In [2]:

def fixDataLength(path):
    '''
    gets rid of the weird length stuff
    path (string) : filepath to the .log file for a participant
    '''     
    x = open(path)
    s = x.read().replace(',,,', ',NaN,NaN,')
    x.close()

    # Now open the file in write mode and write the string with replaced

    x=open(path,'w')
    x.write(s)
    x.close()
    

In [28]:
def getAccAndRT(df):
    '''
    returns [array]: [reaction time, accuracy] for dataframe df
    df needs to have ['respTime'] and ['accuracy'] columns, and a name
    '''
    accCount = df['accuracy'].astype(int).sum()

    accPerc = accCount / len(df)
    RT = df['respTime'].astype(float).mean()
    
    print(df.name + ' AccPerc is ' + str(accPerc))
    print(df.name + ' avg resp time: ' + str(RT))
    
    return [accPerc, RT]



In [3]:
import pandas as pd
import glob
import os
import copy

#global script variables
accCutoff = .65
path = r'/Users/jinjiang-macair/Library/CloudStorage/OneDrive-DukeUniversity/Duke/Fall2022Rotation/Data/contextFlanker/'

#load files from path
allFiles = glob.glob(path + "*.txt")
assignmentID = allFiles
subjectList = []
assignmentList = []
groupDataList = []
aboveAccCutoff = []
accList = []

for file in allFiles:
    # open participant info file
    partInfoFile = open(file,"r").readlines()
    partInfo = [x.strip() for x in partInfoFile]
    
    #pull worker ID and assignment ID from file
    wID = partInfo[2]
    aID = partInfo[1]
    compID = wID.split(':')  #getting worker ID
    compID2 = aID.split(':') #getting assignment ID
    subject = compID[1] #subID
    
    #open log file with the same name as the given particant info file
    logfile = compID2[1]+'.log'
    
    fixDataLength(path+logfile) #dunno if i should return the log file instead of doing a void fxn here
    
    partDataFile = pd.read_csv(path + logfile, lineterminator=';', sep=',', header=None, error_bad_lines = False);
    partData = pd.concat([partDataFile], ignore_index=True, axis = 1) 
    testData = copy.deepcopy(partData)
    
    #formatting result dataframe
    
    # these are longer than they seem.
    partData['subject']=subject #adding a column with subID
    partData = partData.rename(columns={0: 'sectionType', 1: 'taskName', 2: 'blockNumber', 3: 'firstBlockName', 4: 'secondBlockName', 5: 'thirdBlockName', 
                                        6:'currentBlockName', 7: 'trialCount', 8: 'blockTrialCount', 
                                        9: 'trialImg', 10: 'imgSource', 11:'imgCongruency', 13: 'imgDisplayLocation', 13:'repeat', 
                                        14: 'stimOnset', 15: 'respOnset', 16: 'respTime', 17: 'accuracy', 18: 'redundantDispLoc', 19: 'redundantCongruency',
                                        20: 'sectionStart', 21: 'sectionEnd', 22: 'sectionEnd-sectionStart', 23: 'useless1', 24: 'useless2', 25: 'useless3', 26:'subject'}) 
       
    #add info to individual lists
    subjectList.append(subject)
    assignmentList.append(compID2[1])
    
    #add data to whole sample dataframe list
    groupDataList.append(partData)

    #save out participant data to CSV if it doesn't exist already
    if os.path.exists(path+'Subj_'+str(subject)+'.csv'):
        print(subject+' exists.') #don't make csv again if it already exits (file creation times are helpful for keeping track of data)
    else:
        print('Creating csv file for '+subject)
        partData.to_csv(path+'Subj_'+str(subject)+'.csv', index=False)
    
#combine groupData into one file and save file
groupData = pd.concat(groupDataList, ignore_index = True)

# save out groupData into csv
print('Creating group level csv file.')
groupData.to_csv(path+'combinedData'+'.csv', index=False)

a10ij5ugn53ua7 exists.
a28d99x06zi4as exists.
a9mjvjamlcdmv exists.
a3gvuyy3twrpzt exists.
a1o7wvjlbkcx1a exists.
a3su002smrt617 exists.
a2tc7b46wq695o exists.
a1utnak70qh9js exists.
a1rv2lervs0a4h exists.
awsg11rh1py2i exists.
a3cimzpg2oy5zo exists.
a3ljsvb3hlepnw exists.
a1s6rzmvtbgzre exists.
ajabapm32etmn exists.
Creating group level csv file.


b'Skipping line 2: expected 24 fields, saw 76\nSkipping line 3: expected 24 fields, saw 26\nSkipping line 4: expected 24 fields, saw 26\nSkipping line 5: expected 24 fields, saw 26\nSkipping line 6: expected 24 fields, saw 26\nSkipping line 7: expected 24 fields, saw 26\nSkipping line 8: expected 24 fields, saw 26\nSkipping line 9: expected 24 fields, saw 26\nSkipping line 10: expected 24 fields, saw 26\nSkipping line 11: expected 24 fields, saw 26\nSkipping line 12: expected 24 fields, saw 26\nSkipping line 13: expected 24 fields, saw 26\nSkipping line 14: expected 24 fields, saw 26\nSkipping line 15: expected 24 fields, saw 26\nSkipping line 16: expected 24 fields, saw 26\nSkipping line 17: expected 24 fields, saw 26\nSkipping line 18: expected 24 fields, saw 26\nSkipping line 19: expected 24 fields, saw 26\nSkipping line 20: expected 24 fields, saw 26\nSkipping line 21: expected 24 fields, saw 26\nSkipping line 22: expected 24 fields, saw 26\nSkipping line 23: expected 24 fields, sa

In [29]:
path = '/Users/jinjiang-macair/Library/CloudStorage/OneDrive-DukeUniversity/Duke/Fall2022Rotation/ContextCognitiveControlCode/analysis/test2.log'
x = open(path)
s = x.read().replace(';', '\n')
x.close()

# Now open the file in write mode and write the string with replaced

x=open(path,'w')
x.write(s)
x.close()





In [28]:
# find a way to replace all characters in one go cuz this is so dumb
path = '/Users/jinjiang-macair/Library/CloudStorage/OneDrive-DukeUniversity/Duke/Fall2022Rotation/ContextCognitiveControlCode/analysis/test2.log'
x = open(path)
s = x.read().replace('test,,', 'test,test,')
x.close()

# Now open the file in write mode and write the string with replaced

x=open(path,'w')
x.write(s)
x.close()

# also delete all the practice trials cuz their lengths are messed up rn


In [30]:
# find a way to replace all characters in one go cuz this is so dumb
path = '/Users/jinjiang-macair/Library/CloudStorage/OneDrive-DukeUniversity/Duke/Fall2022Rotation/ContextCognitiveControlCode/analysis/test2.log'
x = open(path)
s = x.read().replace(',,,', ',NaN,NaN,')
x.close()

# Now open the file in write mode and write the string with replaced

x=open(path,'w')
x.write(s)
x.close()





In [5]:
allData = pd.read_csv('/Users/jinjiang-macair/Library/CloudStorage/OneDrive-DukeUniversity/Duke/Fall2022Rotation/Data/contextFlanker/combinedData.csv')

In [33]:
print('avg con resp time: ' + str(conDF['respTime'].astype(float).mean()))

print('avg inc resp time: ' + str(incDF['respTime'].astype(float).mean()))

avg con resp time: 1067.6666666666667
avg inc resp time: 1079.307126258714


In [6]:
import pandas as pd

accThreshold = 75

highAccDF = pd.DataFrame()

subjectList.remove('a3ljsvb3hlepnw')

for subject in subjectList:
    subjectDF = allData.loc[allData['subject'] == subject]
    
    '''
    Check if the accuracy of each subject is above our threshold.
    If so, add it to the highAccDF.
    '''
    if (100 * subjectDF['accuracy'].astype(int).sum() / len(subjectDF)) > accThreshold:
        highAccDF = pd.concat([highAccDF, subjectDF])
    




In [22]:
conDF = highAccDF.loc[(highAccDF['imgCongruency'] == 'c')]

incDF = highAccDF.loc[(highAccDF['imgCongruency'] == 'i')]


conAccCount = conDF['accuracy'].astype(int).sum()
incAccCount = incDF['accuracy'].astype(int).sum()



conAccPerc = conAccCount / len(conDF)
incAccPerc = incAccCount / len(incDF)

print('conAccPerc is ' + str(conAccPerc))

print('incAccPerc is: ' + str(incAccPerc))


conAccPerc is 0.8658064516129033
incAccPerc is: 0.8407479045776918


In [24]:
# need to grab the respTime and Acc for each trial AFTER a congruent trial


# make an empty dataframe for trials AFTER congruent ones
# make another empty dataframe for trials after incongruent ones

# for each row in the test block (trialCount is > 200)
    # check if it's congruent or incongruent
        # if congruent
            # grab next row and add to the afterCongruent dataframe
        # if incongruent
            # grab next row and add to the afterIncongruent dataframe
            
testBlockDF = highAccDF.loc[highAccDF['currentBlockName'].isnull()]


afterConDF = testBlockDF.loc[testBlockDF['imgCongruency'].shift(1) == 'c'] # can you just do this
afterIncDF = testBlockDF.loc[testBlockDF['imgCongruency'].shift(1) == 'i']

afterConDF.name = 'afterConDF'
afterIncDF.name = 'afterIncDF'


In [30]:
getAccAndRT(afterConDF)

getAccAndRT(afterIncDF)

afterConDF AccPerc is 0.8829787234042553
afterConDF avg resp time: 1027.4536817102137
afterIncDF AccPerc is 0.8390532544378698
afterIncDF avg resp time: 1036.0640569395018


[0.8390532544378698, 1036.0640569395018]